In [1]:
import pandas as pd
import numpy as np

In [2]:
correct_files = [
                    '../Data/submission_nn_resnet_polar_soft_pl_n_valid_oof_augment_origin.csv',
                    '../Data/submission_nn_resnet_polar_oof_augment_origin.csv',
                    '../Data/submission_09_soft_pl_n_valid_resnet_origin_oof_augment_origin.csv'
                ]
cutoff_hi=.7
cutoff_lo=.3

In [12]:
repeat_count=40
fold_num=5
def correct(file, cutoff_hi=.1, cutoff_lo=.9):
    print('processing: ', file)
    df = pd.read_csv(file)
    
    # the first two columns are 'id' and 'is_iceberg'
    df['is_iceberg_max'] = df.iloc[:, 2:repeat_count*fold_num+2].max(axis=1)
    df['is_iceberg_min'] = df.iloc[:, 2:repeat_count*fold_num+2].min(axis=1)
    df['is_iceberg_mean'] = df.iloc[:, 2:repeat_count*fold_num+2].mean(axis=1)
    df['is_iceberg_median'] = df.iloc[:, 2:repeat_count*fold_num+2].median(axis=1)
    
    df['is_iceberg'] = df['is_iceberg_mean']
    df[['id', 'is_iceberg']].to_csv(file.replace('origin', 'avg'), index=False, float_format='%.15f')
    
    df['is_iceberg'] = np.where(np.all(df.iloc[:,2:repeat_count*fold_num+2] > cutoff_lo, axis=1), 
                                    df['is_iceberg_max'], # all > lo
                                    np.where(np.all(df.iloc[:,2:repeat_count*fold_num+2] < cutoff_hi, axis=1),
                                             df['is_iceberg_min'], # all < hi 
                                             df['is_iceberg_median'] # use the best
                                            )
                                   )
    df[['id', 'is_iceberg']].to_csv(file.replace('origin', 'minmax'), index=False, float_format='%.15f')
    df.to_csv(file, index=False, float_format="%.15f")

In [13]:
for f in correct_files:
    correct(f, cutoff_hi, cutoff_lo)

processing:  ../Data/submission_nn_resnet_polar_soft_pl_n_valid_oof_augment_origin.csv
processing:  ../Data/submission_nn_resnet_polar_oof_augment_origin.csv
processing:  ../Data/submission_09_soft_pl_n_valid_resnet_origin_oof_augment_origin.csv
